In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

Data indicates most wines are in the middle regarding quality

In [93]:
df = pd.read_csv('clean_data.csv')

In [94]:
df.sample(5)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,white,high_quality
4100,4100,6.3,0.29,0.23,14.2,0.037,24.0,99.0,0.99528,3.08,0.38,10.6,6,1,0
4915,4915,8.1,0.56,0.28,1.7,0.368,16.0,56.0,0.99680,3.11,1.28,9.3,5,0,0
3197,3197,7.9,0.34,0.44,6.5,0.027,47.0,126.0,0.99124,2.96,0.37,12.5,6,1,0
2000,2000,6.8,0.31,0.09,1.4,0.040,56.0,145.0,0.99220,3.19,0.46,10.0,5,1,0
1798,1798,6.8,0.23,0.29,15.4,0.073,56.0,173.0,0.99840,3.06,0.41,8.7,6,1,0


In [114]:
#Create our features
X = df.loc[:,df.columns!='high_quality']
X = X.drop("quality",axis=1)
X = X.drop("Unnamed: 0",axis=1)
X = pd.get_dummies(X)

#Create our target
target = ["high_quality"]
y = df["high_quality"]
#y = df.loc[:, target].copy()

In [115]:
X.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.216579,0.339691,0.318722,5.444326,0.056042,30.525319,115.744574,0.994697,3.218395,0.531215,10.491801,0.753886
std,1.295751,0.164548,0.145231,4.757392,0.035031,17.749400,56.521855,0.002999,0.160637,0.148768,1.192712,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,1.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,1.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,1.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,1.000000


In [116]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [117]:
#Check the balance of our target values
y.value_counts()

0    5220
1    1277
Name: high_quality, dtype: int64

In [118]:
#Split the dataset into Test/Training sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [119]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [120]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [121]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [122]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [123]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.8107023024412819

In [124]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[998, 292],
       [ 51, 284]], dtype=int64)

In [125]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.77      0.85      0.85      0.81      0.65      1290
          1       0.49      0.85      0.77      0.62      0.81      0.66       335

avg / total       0.86      0.79      0.83      0.81      0.81      0.65      1625



In [126]:
importances = brf.feature_importances_ 
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[indices[f]]}: ({importances[indices[f]]})")

alcohol: (0.18145027553059162)
density: (0.11415948934641454)
volatile acidity: (0.09713546210528046)
chlorides: (0.08961699487765529)
total sulfur dioxide: (0.08090763161293406)
citric acid: (0.07890735960150713)
residual sugar: (0.0788251226596966)
pH: (0.07484453873050473)
sulphates: (0.0730948038405446)
free sulfur dioxide: (0.0659813034526162)
fixed acidity: (0.062020885150576285)
white: (0.003056133091678595)


In [127]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [128]:
y_pred=rfc.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.7830614370010414

# Hyperparameter Tuning

In [129]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=90, stop=200, num=12)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(start=10, stop=110, num=11)]
max_depth.append(None)
min_samples_split=[2, 5, 10]
min_samples_leaf=[1, 2, 4]
bootstrap=[True, False]

In [130]:
random_search_grid = {'n_estimators': n_estimators,
                      'max_features': max_features,
                      'max_depth': max_depth,
                      'min_samples_split': min_samples_split,
                      'min_samples_leaf': min_samples_leaf,
                      'bootstrap': bootstrap}
print(random_search_grid)

{'n_estimators': [90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [131]:
rfc=RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator=rfc, param_distributions = random_search_grid, n_iter=100, 
                          cv=3, verbose=2, random_state=0, n_jobs=-1)

In [132]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   41.1s finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [90, 100, 110, 120, 130,
                                                         140, 150, 160, 170,
                                                         180, 190, 200]},
                   random_state=0, verbose=2)

In [133]:
rf_random.best_params_

{'n_estimators': 110,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 70,
 'bootstrap': True}

In [134]:
rfc = RandomForestClassifier(n_estimators=90, min_samples_split=2, min_samples_leaf=1, 
                             max_features='auto', max_depth=50, bootstrap=True,
                             random_state = 42)

In [135]:
rfc.fit(X_train,y_train)

RandomForestClassifier(max_depth=50, n_estimators=90, random_state=42)

In [137]:
y_pred=rfc.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.7758127964827027

In [138]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\nClassification Report:\n', classification_report(y_test, y_pred))

Confusion Matrix:
 [[1243   47]
 [ 138  197]]


NameError: name 'classification_report' is not defined